In [2]:
### a. Import the necessary packages

# Import TensorFlow, the main library
import tensorflow as tf
# Import Keras, the high-level API for TensorFlow
from tensorflow import keras
# Import the CIFAR-10 dataset loader
from tensorflow.keras.datasets import cifar10
# Import the Sequential model (a linear stack of layers)
from tensorflow.keras.models import Sequential
# Import the layers we need:
# Dense: A standard, fully-connected layer (for an FNN)
# Flatten: To convert the 3D image into a 1D vector
# Dropout: A regularization technique to prevent overfitting
from tensorflow.keras.layers import Dense, Flatten, Dropout
# Import the 'to_categorical' utility to one-hot encode our labels
from tensorflow.keras.utils import to_categorical
# Import matplotlib for plotting our results
import matplotlib.pyplot as plt
# Import numpy for numerical operations
import numpy as np
import pandas as pd
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.17.0


In [3]:
# ---
### b. Load the training and testing data
# ---

print("Loading CIFAR-10 dataset...")
# Load the dataset. Keras provides this helper function.
# X_train/X_test are the images (50000 train, 10000 test)
# y_train/y_test are the labels (0-9 for the 10 classes)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Define the 10 class names for CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

print(f"X_train shape: {X_train.shape}") # (50000, 32, 32, 3)
print(f"y_train shape: {y_train.shape}") # (50000, 1)

Loading CIFAR-10 dataset...
X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)


In [7]:
# --- Preprocessing ---

# 1. Normalize the pixel values
# The images are 32x32 pixels with 3 color channels (RGB).
# Pixel values are 0-255. We scale them to be between 0.0 and 1.0.
# We convert to 'float32' for the division.
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# 2. One-Hot Encode the labels
# y_train/y_test are arrays of single digits (e.g., [5], [0], [9]).
# We need to convert this to a "one-hot" vector.
# e.g., 5 becomes [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
# The '10' tells the function there are 10 possible classes.
y_train_categorical = to_categorical(y_train, 10)
y_test_categorical = to_categorical(y_test, 10)

print(f"y_train_categorical shape: {y_train_categorical.shape}") # (50000, 10)
print("Data loading and preprocessing complete.")

y_train_categorical shape: (50000, 10)
Data loading and preprocessing complete.


In [4]:
# Initialize a Sequential model
model = Sequential()

# 1. Add the Flatten layer
# This is the most important step for using an FNN.
# It "flattens" the 3D image (32x32x3) into a 1D vector.
# 32 * 32 * 3 = 3072. So, this layer outputs a vector of size 3072.
# We must specify 'input_shape' for the first layer of the model.
model.add(Flatten(input_shape=(32, 32, 3)))

# 2. Add the Hidden Layers
# These are the 'Dense' (fully-connected) layers.
# 'relu' (Rectified Linear Unit) is a standard, effective activation function.
model.add(Dense(512, activation='relu'))
# Add a Dropout layer for regularization.
# It randomly "turns off" 20% of neurons during training
# to prevent the model from "memorizing" the training data (overfitting).
model.add(Dropout(0.2))

# Add a second hidden layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

# 3. Add the Output Layer
# It must have 10 neurons, one for each class.
# 'softmax' activation converts the output into a probability
# distribution (all 10 outputs sum to 1.0). The highest
# probability is the model's prediction.
model.add(Dense(10, activation='softmax'))

# Print a summary of our model
print("\nModel Architecture:")
model.summary()


Model Architecture:


C:\Users\ishan\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 3072)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,573,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,707,274 (6.51 MB)

 Trainable params: 1,707,274 (6.51 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# ---
### d. Train the model using Adam optimizer
# ---

# We'll use 'Adam' as it's a robust, efficient optimizer that
# generally works well with default settings.

# Compile the model
# This configures the model for training.
# 1. optimizer: 'Adam' adjusts the learning rate as it trains.
# 2. loss: 'categorical_crossentropy' is the standard loss function
#    for multi-class classification with one-hot encoded labels.
# 3. metrics: We ask the model to report its 'accuracy' at each step.
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# Set training parameters
BATCH_SIZE = 64  # Process 64 images at a time
EPOCHS = 15      # Go through the entire dataset 15 times

print(f"\nStarting training for {EPOCHS} epochs...")

# Train the model
# model.fit() is the function that does the training.
# We pass it the training data (X_train, y_train_categorical).
# We specify the batch size and number of epochs.
# We pass 'validation_data' (our test set) so Keras
# can evaluate the model on unseen data after each epoch.
history = model.fit(X_train, y_train_categorical,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(X_test, y_test_categorical))

print("Training finished.")


Starting training for 15 epochs...
Epoch 1/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - accuracy: 0.2255 - loss: 2.1303 - val_accuracy: 0.3343 - val_loss: 1.8604
Epoch 2/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.2990 - loss: 1.9026 - val_accuracy: 0.3591 - val_loss: 1.7940
Epoch 3/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 20ms/step - accuracy: 0.3191 - loss: 1.8461 - val_accuracy: 0.3775 - val_loss: 1.7502
Epoch 4/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 31s 20ms/step - accuracy: 0.3328 - loss: 1.8181 - val_accuracy: 0.3805 - val_loss: 1.7221
Epoch 5/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.3438 - loss: 1.7939 - val_accuracy: 0.3957 - val_loss: 1.7107
Epoch 6/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.3477 - loss: 1.7788 - val_accuracy: 0.3976 - val_loss: 1.6784
Epoch 7/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.3522 - loss: 1.7712 - val_accuracy: 0.4045 - val_loss: 1.6817
Epoch 8/15
1563/1563 ━━━━━━━━━━━━━━━━━━━